# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'ND0002'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 35.5 s, sys: 3.67 s, total: 39.2 s
Wall time: 40.8 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0103K1F1P1R1,Ok,r01c03f01p01-ch1sk1fk1fl1.tiff,1,3,1,1,0,1,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.49+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
1,0103K1F1P1R2,Ok,r01c03f01p01-ch2sk1fk1fl1.tiff,1,3,1,1,0,2,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.723+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
2,0103K1F1P2R1,Ok,r01c03f01p02-ch1sk1fk1fl1.tiff,1,3,1,2,0,1,1,...,0,0.135468394,0,2023-11-30T17:22:10.067+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
3,0103K1F1P2R2,Ok,r01c03f01p02-ch2sk1fk1fl1.tiff,1,3,1,2,0,2,1,...,0,0.135468394,0,2023-11-30T17:22:10.287+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
4,0103K1F1P3R1,Ok,r01c03f01p03-ch1sk1fk1fl1.tiff,1,3,1,3,0,1,1,...,2E-06,0.135470405,0,2023-11-30T17:22:10.627+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388615,0612K150F9P1R2,Ok,r06c12f09p01-ch2sk150fk1fl1.tiff,6,12,9,1,149,2,1,...,-2E-06,0.1351538,268191.66,2023-12-03T20:06:16.08+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388616,0612K150F9P2R1,Ok,r06c12f09p02-ch1sk150fk1fl1.tiff,6,12,9,2,149,1,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.423+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388617,0612K150F9P2R2,Ok,r06c12f09p02-ch2sk150fk1fl1.tiff,6,12,9,2,149,2,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.657+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388618,0612K150F9P3R1,Ok,r06c12f09p03-ch1sk150fk1fl1.tiff,6,12,9,3,149,1,1,...,2E-06,0.135157794,268191.66,2023-12-03T20:06:17+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

In [4]:
acq_ID = row, column = (3, 4)

#### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [ ]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

#### Load images using Zarr (WIP)

In [ ]:
import zarr

In [ ]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
images = zarr.open(image_dir, mode='r')

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [20]:
from importlib import reload

In [72]:
reload(tile)

<module 'macrohet.tile' from '/home/dayn/analysis/macrohet/macrohet/tile.py'>

In [73]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
dask_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
dask_images

<class 'list'>
<class 'dask.array.core.Array'>
CPU times: user 2.95 s, sys: 57.6 ms, total: 3.01 s
Wall time: 2.37 s


dask.array<max-aggregate, shape=(1, 2, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [68]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
dask_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
dask_images

<class 'list'>
<class 'dask.array.core.Array'>
CPU times: user 424 ms, sys: 3.83 ms, total: 428 ms
Wall time: 424 ms


dask.array<max-aggregate, shape=(1, 2, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2160, 2160), chunktype=numpy.ndarray>

In [75]:
%%timeit
images = dask_images.compute()

5.96 s ± 84.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
%%timeit
images = dask_images.compute()

5.99 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
%%timeit
images = dask_images.compute()

9 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%%timeit
images = dask_images.compute()

8.91 s ± 89.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%%timeit
images = dask_images.compute()

9.03 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
images

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [114, 117, 108, ..., 113, 115, 118],
         [114, 112, 100, ..., 116, 113, 120],
         ...,
         [100, 101, 112, ..., 116, 114, 107],
         [102, 104, 105, ..., 112, 118, 115],
         [110, 104, 104, ..., 117, 113, 111]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [134, 131, 131, ..., 124, 111, 100],
         [128, 119, 134, ..., 118, 115, 104],
         ...,
         [281, 291, 313, ..., 123, 115, 107],
         [250, 275, 287, ..., 126, 122, 109],
         [243, 254, 249, ..., 120, 114, 110]]]], dtype=uint16)

In [47]:
%%timeit
images = dask_images.compute()

6.21 s ± 96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%timeit
images = dask_images.compute()

6.11 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}')

viewer.add_image(images, channel_axis = 1, 
                 # scale = napari_scale, 
                 contrast_limits=[[280, 1000],[0,3000]])

[<Image layer 'Image' at 0x7f651a384d90>,
 <Image layer 'Image [1]' at 0x7f651a2d6d00>]